In [ ]:

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt  # Para graficar la FFT
import quantstats as qs
from itertools import product
import logging
import matplotlib.font_manager
# Ignorar advertencias de métricas no definidas

# Lista de resultados
pattern_results = []
pct_pattern_results = []  # resultados solo de patrones percentiles/threshold
# Rango de porcentajes a evaluar
pct_thresholds = [round(x, 2) for x in list(np.arange(0.1, 2.01, 0.1))]

# Inicializa QuantBook
qb = QuantBook()

all_best_combinations = []
all_best_seasons = []
count = 0
for symbol in symbols:
    count += 1
    print(count, symbol)
    asset = qb.AddEquity(symbol, Resolution.HOUR, dataNormalizationMode=DataNormalizationMode.RAW).Symbol
    df = qb.History(asset, start=datetime(1997, 1, 1), end=qb.time, resolution=Resolution.HOUR)
    if isinstance(df.index, pd.MultiIndex):
        df = df.droplevel([0])
    if not df.empty:
        if df.index.min().year > 2007 or df.index.max().year < 2025:
            print(f"Datos insuficientes para {symbol}, descartando...")
            continue
        df = df[['open', 'high', 'low', 'close', 'volume']].copy()
        df['returns_pct'] = ((df['close'] - df['open']) / df['open']) * 100
        # Evaluar patrones fijos
        trend_patterns = {
            "up_trend": lambda df: df['open'].shift(1) > df['close'].shift(1),
            "down_trend": lambda df: df['open'].shift(1) < df['close'].shift(1)
        }
        for pattern_name, pattern_func in trend_patterns.items():
            pattern_filter = pattern_func(df)
            filtro = df[pattern_filter]
            if filtro.empty:
                continue
            pnls = filtro['returns_pct']
            num_trades = len(pnls)
            gains = pnls[pnls > 0].sum()
            losses = abs(pnls[pnls < 0].sum())
            pf_longs = gains / losses if losses != 0 else float('inf')
            sharpe_ratio = qs.stats.sharpe(pnls) if len(pnls) > 1 else float('inf')
            pattern_results.append([
                symbol, pattern_name, None, pf_longs, sharpe_ratio, num_trades
            ])
        # Evaluar rangos de thresholds para up_pct y down_pct
        for threshold in pct_thresholds:
            up_pattern_name = "up_pct"
            down_pattern_name = "down_pct"
            up_filter = df['returns_pct'].shift(1) > threshold
            down_filter = df['returns_pct'].shift(1) < -threshold
            for pattern_name, pattern_filter in [(up_pattern_name, up_filter), (down_pattern_name, down_filter)]:
                filtro = df[pattern_filter]
                if filtro.empty:
                    continue
                pnls = filtro['returns_pct']
                num_trades = len(pnls)
                gains = pnls[pnls > 0].sum()
                losses = abs(pnls[pnls < 0].sum())
                pf_longs = gains / losses if losses != 0 else float('inf')
                sharpe_ratio = qs.stats.sharpe(pnls) if len(pnls) > 1 else float('inf')
                pct_pattern_results.append([
                    symbol, pattern_name, threshold, pf_longs, sharpe_ratio, num_trades
                ])
    else:
        print(f"No hay datos disponibles para {symbol}")
# Crear DataFrames finales
results_df = pd.DataFrame(pattern_results, columns=[
    "Symbol", "Pattern", "Value", "ProfitFactor", "SharpeRatio", "NumTrades"
])
results_df = results_df.sort_values(by=["ProfitFactor"], ascending=False)

results_df_pct = pd.DataFrame(pct_pattern_results, columns=[
    "Symbol", "Pattern", "Value", "ProfitFactor", "SharpeRatio", "NumTrades"
])
results_df_pct = results_df_pct.sort_values(by=["ProfitFactor"], ascending=False)
# Apagar conexión



In [ ]:

filtered_df = results_df[
    (results_df["NumTrades"] > 1000) &
    (results_df["ProfitFactor"] > 1.3) &
    (results_df["SharpeRatio"] > 1.3)
    #(results_df["Pattern"].str.contains("up_trend"))
]
display(filtered_df.head(50))


In [ ]:

filtered_df_pct = results_df_pct[
    (results_df_pct["NumTrades"] > 100) &
    (results_df_pct["ProfitFactor"] > 1.5) &
    (results_df_pct["SharpeRatio"] > 1.5)
    #(results_df_pct["Pattern"].str.contains("up_pct"))
]
display(filtered_df_pct.head(50))


In [ ]:

import logging
import matplotlib.font_manager
import pandas as pd
import matplotlib.pyplot as plt
import quantstats as qs

logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

long = True

# Iterar sobre cada combinación filtrada en orden cronológico
for index, row in results_df_pct.iterrows():
    symbol = row["Symbol"]
    Pattern = row["Pattern"]
    Value = row["Value"]

    asset = qb.AddEquity(symbol, Resolution.HOUR, dataNormalizationMode=DataNormalizationMode.RAW).Symbol
    history = qb.History(asset, start=datetime(1997, 1, 1), end=qb.time, resolution=Resolution.HOUR)

    # Verificar si hay datos antes de continuar
    history = history.droplevel([0])

    if not history.empty:
        history = history[['open', 'high', 'low', 'close', 'volume']].copy()

        history['returns_pct'] = ((history['close'] - history['open']) / history['open']) * 100
        history['pct_range'] = abs((history['high'] - history['low']) / history['low']) * 100

        if not long:
            history['returns_pct'] *= -1  # Invertir retornos para shorts

        history['month'] = history.index.get_level_values('time').month
        history['day'] = history.index.get_level_values('time').day
        history['weekday'] = history.index.get_level_values('time').weekday
        history['hour'] = history.index.get_level_values('time').hour
        up_trend = (history['close'].shift(1) > history['close'].shift(2))
        down_trend = (history['close'].shift(1) < history['close'].shift(2))
        up_pct = history['returns_pct'] > Value
        down_pct = history['returns_pct'] < Value

        # Filtrar datos según los valores de weekday y hour de la combinación actual
        filtro = history[(up_pct)].copy()

        # Guardar los retornos filtrados en una serie de pandas
        pnls = pd.Series(filtro['returns_pct'].values, index=filtro.index) / 100
        filtro['equity'] = (1 + filtro['returns_pct'] / 100).cumprod() * 100

        # Generar informe de estrategia
        qs.extend_pandas()
        pnls.plot_snapshot(title=f'Rendimiento de la Estrategia ({symbol}, Weekday {weekday}, Hour {hour})', show=True)

        num_trades = len(pnls)
        print("\n" + "="*80)
        print(f"PERFORMANCE REPORT: {symbol} (Weekday {weekday}, Hour {hour})")
        print("="*80 + "\n")
        print(f"{'Metric':<30}{'Value':>20}")
        print("-"*50)
        print(f"{'Profit Factor':<30}{qs.stats.profit_factor(pnls):>20.2f}")
        print(f"{'Annualized Return (CAGR)':<30}{qs.stats.cagr(pnls):>20.2%}")
        print(f"{'Sharpe Ratio':<30}{qs.stats.sharpe(pnls):>20.2f}")
        print(f"{'Sortino Ratio':<30}{qs.stats.sortino(pnls):>20.2f}")
        print(f"{'Max Drawdown':<30}{qs.stats.max_drawdown(pnls):>20.2%}")
        print(f"{'Value at Risk (VaR 95%)':<30}{qs.stats.value_at_risk(pnls):>20.2%}")
        print(f"{'Conditional VaR (CVaR 95%)':<30}{qs.stats.conditional_value_at_risk(pnls):>20.2%}")
        print(f"{'Recovery Factor':<30}{qs.stats.recovery_factor(pnls):>20.2f}")
        print(f"{'Win Rate':<30}{qs.stats.win_rate(pnls):>20.2%}")
        print(f"{'Skewness':<30}{qs.stats.skew(pnls):>20.2f}")
        print(f"{'Kurtosis':<30}{qs.stats.kurtosis(pnls):>20.2f}")
        print(f"{'Calmar Ratio':<30}{qs.stats.cagr(pnls) / abs(qs.stats.max_drawdown(pnls)):>20.2f}")
        print(f"{'Number of Trades':<30}{num_trades:>20}")
        print("-"*50 + "\n")
        print("="*80 + "\n")
